# Learning topological operations on meshes with application to block decomposition of polygons

A.Narayanan, Y.Pan, P.-O. Persson

**Objectif :** Améliorer la régularité des maillages, en se concentrant sur la connectivité des maillages. L’idée est de minimiser les irrégularités sur les sommets.

Ce papier se base sur des études de maillages 2D triangulaires et quadrangulaires.  Dans la suite sera traitée uniquement la partie maillage triangulaires.

Les nœuds d’un maillage disposent d’un certain nombre d’arêtes incidentes. Ceci est une propriété principale de la régularité d’un maillage. En effet, chaque sommet a un nombre d’arêtes incidentes idéal.


- Pour les nœuds internes :  $360/60° = 6$
- Pour les noeuds externes : $ max ([\theta/\alpha] +1, 2) = 2,3,4 \dots $

Si ce nombre n’est pas atteint, il existe alors des irrégularités. L’objectif étant d’avoir le moins d’irrégularités possibles, un global score peut alors être défini :

* $s= \sum_1^{N_v} | \Delta i | $  

* $s*= | \sum_1^{N_v} \Delta i | $

De plus, afin d’améliorer la connectivité, dans le cas des maillages triangulaires il existes trois actions possibles :
- Edge Flip 
- Edge Split
- Edge Collapse



Afin de représenter ces maillages, la structure **DCEL** (doubly connected edge list) est utilisée.

Afin de générer une base de maillage à étudier, les méthodes « Delaunay refinement » sont appliquées. Ces dernières ne permettent pas de définir des structures de connectivités spécifiques. C’est pourquoi, ce travail cherche à améliorer la connectivité des ces maillages en utilisant un algorithme d’apprentissage par renforcement.

Le problème peut être modélisé comme un **Markov Decision Process**, il s’agit d’une modélisation adaptée lorsque l’environnement est connu (nombre fini d’état, d’actions possibles, et de récompenses).
Selon l’état dans lequel on se trouve, une action différente sera choisie et donc une récompense différente.  
MDP involve delayed reward ⇒ il faut donc gérer les récompenses immédiates et futures.
Ce qui peut se modéliser comme suit :

In [6]:
from IPython.display import Image
Image(url="Schema.jpg", width=300, height=200)

Après chaque action, la récompense obtenue sera : $rt = st-st+1$

Lorsque le maillage a subit t tranformations et est alors à l’état Mt, le discounted return est defini comme suit :

$Gt = \sum_{k=t}^{n} \gamma^{k-t} r_k  $

Le « jeu » s’arrête lorsque $st=s*$, c’est à dire que le maillage idéal est atteint, ou que le nombre de pas maximal a été atteint.

Pour entraîner le modèle, l’algorithme de Proximal Policy Optimization est utilisé. Il s’agit d’un algorithme on policy. Classé comme une policy gradient methode, il s’agit d’une version sophistiquée de la méthode REINFORCE

Opération de **CONVOLUTION** 

Les opérations de convolutions permettent d'encoder les informations topologiques autour de chaque brin. En répétant ces opérations, les informations topologiques de tous le maillage sont encodées, et ce de manière expensive.

L'état de chaque brin permet de représenter la topologie locale.

SCHEMA

Les caractéristiques topologiques de chaque brin sont ensuite stockées dans une matrice caractéristique x :

SCHEMA 



**Résultats :**

Le modèle a été entraîné sur 100 rollouts, de plus, les performances ont été mesurées sur 100 maillages.
Tout d’abord ils ont obtenu une performance de 0.81 (sigma=0.11). Mais comme la politique suivie est stochastique (basée sur des probabilités) chaque maillage généré sera différent. Il convient alors de choisir le meilleur maillage après k runs. Avec cette dernière méthode en fixant k=10, les performances s’améliorent: 0.86 (sigma=0.08).

A noter que le temps de convergence à la bonne politique, et  pour atteindre un bon maillage augmente avec la taille des maillages. Cela est à prendre en compte dans la lecture des résultats.

**Questions :**

A quoi correspond la advantage function ? Elle est équivalente à la fonction de valeur d’état v qui est aussi commune en apprentissage par renforcement.

Pourquoi utiliser un « discounted return » et considérer la tâche comme continue et non pas comme des épisodes ? Pour plus prendre en compte les dernières actions.

Qu’est ce que la topologie ?
Comment sont construits exactement les templates ?

Faire des recherches sur Monte Carlo Search Tree ⇒ Dans le book de RL e revoir les notions de loss functions, et d’entropy